# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import math
import numpy as np
import pandas as pd
import requests
from scipy.stats import percentileofscore as score
from statistics import mean
import xlsxwriter
import yfinance as yf

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [3]:
def get_sp500_tickers():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    wiki_tables = pd.read_html(url)
    symbols = wiki_tables[0]['Symbol']
    return [ticker.replace(".","-") for ticker in symbols.tolist()]
print(get_sp500_tickers()) 

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'AON', 'APA', 'APO', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BAX', 'BDX', 'BRK-B', 'BBY', 'TECH', 'BIIB', 'BLK', 'BX', 'XYZ', 'BK', 'BA', 'BKNG', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF-B', 'BLDR', 'BG', 'BXP', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CAT', 'CBOE', 'CBRE', 'CDW', 'COR', 'CNC', 'CNP', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'COIN', 'CL', 'CMCSA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CRWD', 'CCI', 'CSX', 'CMI', 'CVS', 'DHR', 'DRI', 'DDOG', '

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

(I also include the price and EPS here too)

In [4]:
def get_ticker_data(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        stock_price = info['currentPrice']

        eps = info.get('forwardEps')
        if eps and stock_price:
            forward_pe = round(stock_price / eps, 2)
            pe = forward_pe
        else:
            return None

        return {
            'Ticker': ticker,
            'Stock Price': stock_price,
            'P/E Ratio': pe,
            'Number of Shares to Buy': 'N/A'
        }
    except Exception as e:
        print(f'Error fetching {ticker}: {str(e)}')
        return None
get_ticker_data('AAPL')

{'Ticker': 'AAPL',
 'Stock Price': 209.05,
 'P/E Ratio': 25.16,
 'Number of Shares to Buy': 'N/A'}

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [5]:
ticker_list = get_sp500_tickers()
stock_data = []

for ticker in ticker_list[0:60]:
    stock = get_ticker_data(ticker) 
    stock_data.append(stock)
print(stock_data)

[{'Ticker': 'MMM', 'Stock Price': 146.62, 'P/E Ratio': 18.56, 'Number of Shares to Buy': 'N/A'}, {'Ticker': 'AOS', 'Stock Price': 70.83, 'P/E Ratio': 17.4, 'Number of Shares to Buy': 'N/A'}, {'Ticker': 'ABT', 'Stock Price': 128.55, 'P/E Ratio': 24.91, 'Number of Shares to Buy': 'N/A'}, {'Ticker': 'ABBV', 'Stock Price': 189.31, 'P/E Ratio': 15.61, 'Number of Shares to Buy': 'N/A'}, {'Ticker': 'ACN', 'Stock Price': 274.0, 'P/E Ratio': 19.47, 'Number of Shares to Buy': 'N/A'}, {'Ticker': 'ADBE', 'Stock Price': 364.0, 'P/E Ratio': 17.71, 'Number of Shares to Buy': 'N/A'}, {'Ticker': 'AMD', 'Stock Price': 179.51, 'P/E Ratio': 35.2, 'Number of Shares to Buy': 'N/A'}, {'Ticker': 'AES', 'Stock Price': 13.08, 'P/E Ratio': 6.44, 'Number of Shares to Buy': 'N/A'}, {'Ticker': 'AFL', 'Stock Price': 98.59, 'P/E Ratio': 13.75, 'Number of Shares to Buy': 'N/A'}, {'Ticker': 'A', 'Stock Price': 118.05, 'P/E Ratio': 20.89, 'Number of Shares to Buy': 'N/A'}, {'Ticker': 'APD', 'Stock Price': 290.13, 'P/E R

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

In [6]:
pe_columns = ['Ticker', 'Stock Price', 'P/E Ratio', 'Number of Shares to Buy']
pe_df = pd.DataFrame(stock_data, columns=pe_columns)

## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
pe_df.sort_values('P/E Ratio', ascending=True, inplace=True)
pe_df.reset_index(inplace=True, drop=True)
pe_df = pe_df.head(50)
pe_df

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy
0,AES,13.08,6.44,N/A
1,APA,19.48,7.08,N/A
2,ACGL,85.61,9.47,N/A
3,APTV,66.68,9.55,N/A
4,BAX,28.05,9.74,N/A
5,ALL,192.28,10.10,N/A
6,AIZ,184.56,10.30,N/A
7,ADM,55.20,10.89,N/A
8,MO,61.50,11.50,N/A
9,BDX,181.54,11.54,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_input()

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [10]:
position_size = float(portfolio_size) / len(pe_df.index)
for row in pe_df.index:
    pe_df.loc[row, "Number of Shares to Buy"] = math.floor(position_size / pe_df.loc[row, "Stock Price"])
pe_df

,Ticker,Stock Price,P/E Ratio,Number of Shares to Buy
0,AES,13.08,6.44,4587155
1,APA,19.48,7.08,3080082
2,ACGL,85.61,9.47,700852
3,APTV,66.68,9.55,899820
4,BAX,28.05,9.74,2139037
5,ALL,192.28,10.10,312044
6,AIZ,184.56,10.30,325097
7,ADM,55.20,10.89,1086956
8,MO,61.50,11.50,975609
9,BDX,181.54,11.54,330505


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [11]:
def get_more_ticker_data(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        
        stock_price = info['currentPrice']

        eps = info.get('forwardEps')
        if eps and stock_price:
            forward_pe = stock_price / eps
            pe = forward_pe
        else:
            print("Data not available")

        ps = info.get('priceToSalesTrailing12Months', None)


        pb = info.get('priceToBook', None)

        ev_to_ebitda = info.get('enterpriseToEbitda', None) 

        enterprise_value = info.get('enterpriseValue', None)
        gross_profit = info.get('grossProfits', None)
        
        if enterprise_value and gross_profit and gross_profit != 0:
            ev_to_gp = enterprise_value / gross_profit
        else:
            ev_to_gp = None
         
        return {
            'Ticker': ticker,
            'Stock Price': stock_price,
            'P/E Ratio': pe,
            'P/E Percentile': 'N/A',
            'P/S Ratio': ps,
            'P/S Percentile': 'N/A',
            'P/B Ratio': pb,
            'P/B Percentile': 'N/A',
            'EV/EBITDA': ev_to_ebitda,
            'EV/EBITDA Percentile': 'N/A',
            'EV/Gross Profit (EV/GP)': ev_to_gp,
            'EV/GP Percentile': 'N/A',
            'RV Score': 'N/A',
            'Number of Shares to Buy': 'N/A' 
        }
    except Exception as e:
        print(f'Error fetching {ticker}: {str(e)}')
        return None
get_more_ticker_data('AAPL')

{'Ticker': 'AAPL',
 'Stock Price': 209.05,
 'P/E Ratio': 25.156438026474127,
 'P/E Percentile': 'N/A',
 'P/S Ratio': 7.7986865,
 'P/S Percentile': 'N/A',
 'P/B Ratio': 46.756878,
 'P/B Percentile': 'N/A',
 'EV/EBITDA': 23.081,
 'EV/EBITDA Percentile': 'N/A',
 'EV/Gross Profit (EV/GP)': 17.167632380046236,
 'EV/GP Percentile': 'N/A',
 'RV Score': 'N/A',
 'Number of Shares to Buy': 'N/A'}

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [12]:
ticker_list = get_sp500_tickers()
stock_data = []

for ticker in ticker_list[0:60]:
    stock = get_more_ticker_data(ticker) 
    stock_data.append(stock)
print(stock_data)

[{'Ticker': 'MMM', 'Stock Price': 146.62, 'P/E Ratio': 18.559493670886077, 'P/E Percentile': 'N/A', 'P/S Ratio': 3.1743035, 'P/S Percentile': 'N/A', 'P/B Ratio': 18.204618, 'P/B Percentile': 'N/A', 'EV/EBITDA': 15.613, 'EV/EBITDA Percentile': 'N/A', 'EV/Gross Profit (EV/GP)': 8.94570542149993, 'EV/GP Percentile': 'N/A', 'RV Score': 'N/A', 'Number of Shares to Buy': 'N/A'}, {'Ticker': 'AOS', 'Stock Price': 70.83, 'P/E Ratio': 17.4029484029484, 'P/E Percentile': 'N/A', 'P/S Ratio': 2.618703, 'P/S Percentile': 'N/A', 'P/B Ratio': 5.3769073, 'P/B Percentile': 'N/A', 'EV/EBITDA': 26.341, 'EV/EBITDA Percentile': 'N/A', 'EV/Gross Profit (EV/GP)': 13.973869046782369, 'EV/GP Percentile': 'N/A', 'RV Score': 'N/A', 'Number of Shares to Buy': 'N/A'}, {'Ticker': 'ABT', 'Stock Price': 128.55, 'P/E Ratio': 24.91279069767442, 'P/E Percentile': 'N/A', 'P/S Ratio': 5.1881614, 'P/S Percentile': 'N/A', 'P/B Ratio': 4.582071, 'P/B Percentile': 'N/A', 'EV/EBITDA': 20.304, 'EV/EBITDA Percentile': 'N/A', 'EV/

In [13]:
rv_columns = [
    'Ticker',
    'Stock Price',
    'P/E Ratio',
    'P/E Percentile',
    'P/S Ratio',
    'P/S Percentile',
    'P/B Ratio',
    'P/B Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/Gross Profit (EV/GP)',
    'EV/GP Percentile',
    'RV Score',
    'Number of Shares to Buy'
]
rv_df = pd.DataFrame(stock_data, columns=rv_columns)
rv_df

,Ticker,Stock Price,P/E Ratio,P/E Percentile,P/S Ratio,P/S Percentile,P/B Ratio,P/B Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Gross Profit (EV/GP),EV/GP Percentile,RV Score,Number of Shares to Buy
0,MMM,146.62,18.559494,N/A,3.174304,N/A,18.204618,N/A,15.613,N/A,8.945705,N/A,N/A,N/A
1,AOS,70.83,17.402948,N/A,2.618703,N/A,5.376907,N/A,26.341,N/A,13.973869,N/A,N/A,N/A
2,ABT,128.55,24.912791,N/A,5.188161,N/A,4.582071,N/A,20.304,N/A,9.474106,N/A,N/A,N/A
3,ABBV,189.31,15.606760,N/A,5.829086,N/A,235.460190,N/A,14.714,N/A,9.883934,N/A,N/A,N/A
4,ACN,274.00,19.474058,N/A,2.492047,N/A,5.584429,N/A,14.721,N/A,7.888029,N/A,N/A,N/A
5,ADBE,364.00,17.712895,N/A,6.831945,N/A,13.577023,N/A,17.798,N/A,7.838924,N/A,N/A,N/A
6,AMD,179.51,35.198039,N/A,10.488560,N/A,5.011866,N/A,48.507,N/A,19.176954,N/A,N/A,N/A
7,AES,13.08,6.443350,N/A,0.768376,N/A,2.685280,N/A,13.919,N/A,20.257869,N/A,N/A,N/A
8,AFL,98.59,13.750349,N/A,3.156030,N/A,2.030690,N/A,12.553,N/A,7.590277,N/A,N/A,N/A
9,A,118.05,20.893805,N/A,5.059426,N/A,5.462496,N/A,19.749,N/A,10.205665,N/A,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [14]:
rv_df[rv_df.isnull().any(axis=1)]

,Ticker,Stock Price,P/E Ratio,P/E Percentile,P/S Ratio,P/S Percentile,P/B Ratio,P/B Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Gross Profit (EV/GP),EV/GP Percentile,RV Score,Number of Shares to Buy
26,AXP,303.41,20.066799,N/A,3.328408,N/A,6.535628,N/A,NaN,N/A,5.340903,N/A,N/A,N/A
30,AMP,519.75,13.454569,N/A,2.698812,N/A,8.067144,N/A,NaN,N/A,4.335066,N/A,N/A,N/A
37,APO,145.46,17.153302,N/A,3.408764,N/A,5.005162,N/A,NaN,N/A,9.685353,N/A,N/A,N/A
56,BAC,47.96,13.103825,N/A,3.622601,N/A,1.291782,N/A,NaN,N/A,1.579105,N/A,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [15]:
rv_df.dropna(subset=['EV/EBITDA'], inplace=True)
rv_df.reset_index(inplace=True, drop=True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [16]:
rv_df[rv_df.isnull().any(axis=1)]

,Ticker,Stock Price,P/E Ratio,P/E Percentile,P/S Ratio,P/S Percentile,P/B Ratio,P/B Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Gross Profit (EV/GP),EV/GP Percentile,RV Score,Number of Shares to Buy


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [17]:
metrics = { 
    'P/E Ratio': 'P/E Percentile',
    'P/S Ratio': 'P/S Percentile',
    'P/B Ratio': 'P/B Percentile',
    'EV/EBITDA':'EV/EBITDA Percentile',
    'EV/Gross Profit (EV/GP)': 'EV/GP Percentile'
}

In [18]:
for column in metrics.keys():
    for row in rv_df.index:
        rv_df.loc[row, metrics[column]] = score(rv_df[column], rv_df.loc[row, column])
rv_df

,Ticker,Stock Price,P/E Ratio,P/E Percentile,P/S Ratio,P/S Percentile,P/B Ratio,P/B Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Gross Profit (EV/GP),EV/GP Percentile,RV Score,Number of Shares to Buy
0,MMM,146.62,18.559494,42.857143,3.174304,41.071429,18.204618,87.5,15.613,48.214286,8.945705,42.857143,N/A,N/A
1,AOS,70.83,17.402948,37.5,2.618703,32.142857,5.376907,62.5,26.341,87.5,13.973869,71.428571,N/A,N/A
2,ABT,128.55,24.912791,76.785714,5.188161,62.5,4.582071,58.928571,20.304,76.785714,9.474106,50.0,N/A,N/A
3,ABBV,189.31,15.606760,32.142857,5.829086,71.428571,235.460190,100.0,14.714,41.071429,9.883934,51.785714,N/A,N/A
4,ACN,274.00,19.474058,46.428571,2.492047,28.571429,5.584429,66.071429,14.721,42.857143,7.888029,35.714286,N/A,N/A
5,ADBE,364.00,17.712895,39.285714,6.831945,82.142857,13.577023,83.928571,17.798,64.285714,7.838924,28.571429,N/A,N/A
6,AMD,179.51,35.198039,91.071429,10.488560,94.642857,5.011866,60.714286,48.507,96.428571,19.176954,83.928571,N/A,N/A
7,AES,13.08,6.443350,1.785714,0.768376,7.142857,2.685280,44.642857,13.919,35.714286,20.257869,91.071429,N/A,N/A
8,AFL,98.59,13.750349,28.571429,3.156030,39.285714,2.030690,25.0,12.553,26.785714,7.590277,25.0,N/A,N/A
9,A,118.05,20.893805,58.928571,5.059426,58.928571,5.462496,64.285714,19.749,71.428571,10.205665,53.571429,N/A,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [19]:
for row in rv_df.index:
    values = [rv_df.loc[row, metrics[key]] for key in metrics.keys()]
    rv_df.loc[row,'RV Score'] = mean(values)
rv_df

,Ticker,Stock Price,P/E Ratio,P/E Percentile,P/S Ratio,P/S Percentile,P/B Ratio,P/B Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Gross Profit (EV/GP),EV/GP Percentile,RV Score,Number of Shares to Buy
0,MMM,146.62,18.559494,42.857143,3.174304,41.071429,18.204618,87.5,15.613,48.214286,8.945705,42.857143,52.5,N/A
1,AOS,70.83,17.402948,37.5,2.618703,32.142857,5.376907,62.5,26.341,87.5,13.973869,71.428571,58.214286,N/A
2,ABT,128.55,24.912791,76.785714,5.188161,62.5,4.582071,58.928571,20.304,76.785714,9.474106,50.0,65.0,N/A
3,ABBV,189.31,15.606760,32.142857,5.829086,71.428571,235.460190,100.0,14.714,41.071429,9.883934,51.785714,59.285714,N/A
4,ACN,274.00,19.474058,46.428571,2.492047,28.571429,5.584429,66.071429,14.721,42.857143,7.888029,35.714286,43.928571,N/A
5,ADBE,364.00,17.712895,39.285714,6.831945,82.142857,13.577023,83.928571,17.798,64.285714,7.838924,28.571429,59.642857,N/A
6,AMD,179.51,35.198039,91.071429,10.488560,94.642857,5.011866,60.714286,48.507,96.428571,19.176954,83.928571,85.357143,N/A
7,AES,13.08,6.443350,1.785714,0.768376,7.142857,2.685280,44.642857,13.919,35.714286,20.257869,91.071429,36.071429,N/A
8,AFL,98.59,13.750349,28.571429,3.156030,39.285714,2.030690,25.0,12.553,26.785714,7.590277,25.0,28.928571,N/A
9,A,118.05,20.893805,58.928571,5.059426,58.928571,5.462496,64.285714,19.749,71.428571,10.205665,53.571429,61.428571,N/A


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [20]:
rv_df.sort_values('RV Score', inplace=True, ascending=False)
rv_df = rv_df.head(50)
rv_df.reset_index(inplace=True, drop=True)
rv_df

,Ticker,Stock Price,P/E Ratio,P/E Percentile,P/S Ratio,P/S Percentile,P/B Ratio,P/B Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Gross Profit (EV/GP),EV/GP Percentile,RV Score,Number of Shares to Buy
0,AXON,748.60,120.741935,100.0,26.177702,100.0,22.801619,91.071429,499.130,100.0,41.815055,98.214286,97.857143,N/A
1,AMT,208.74,30.742268,87.5,9.524653,91.071429,26.322826,96.428571,22.506,82.142857,19.714529,87.5,88.928571,N/A
2,APH,105.49,48.837963,98.214286,6.844237,83.928571,11.173604,82.142857,25.587,85.714286,20.131747,89.285714,87.857143,N/A
3,ADSK,308.14,33.566449,89.285714,10.386800,92.857143,25.197483,92.857143,44.405,94.642857,11.323607,62.5,86.428571,N/A
4,AAPL,209.05,25.156438,78.571429,7.798686,87.5,46.756878,98.214286,23.081,83.928571,17.167632,80.357143,85.714286,N/A
5,AMD,179.51,35.198039,91.071429,10.488560,94.642857,5.011866,60.714286,48.507,96.428571,19.176954,83.928571,85.357143,N/A
6,ADI,231.11,30.651194,85.714286,11.680609,96.428571,3.275786,48.214286,27.305,89.285714,20.687513,92.857143,82.5,N/A
7,ANET,122.09,12.547790,26.785714,20.619152,98.214286,15.160809,85.714286,44.193,92.857143,29.548407,96.428571,80.0,N/A
8,ADP,310.94,28.605336,82.142857,6.247597,75.0,21.576572,89.285714,21.343,80.357143,13.000972,69.642857,79.285714,N/A
9,AVB,196.31,35.435018,92.857143,9.351211,89.285714,2.342490,33.928571,20.432,78.571429,19.610342,85.714286,76.071429,N/A


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [21]:
portfolio_input()
position_size = float(portfolio_size) / len(rv_df.index)

In [22]:
for row in rv_df.index:
    rv_df.loc[row, 'Number of Shares to Buy'] = round(position_size // rv_df.loc[row, 'Stock Price'])
rv_df

,Ticker,Stock Price,P/E Ratio,P/E Percentile,P/S Ratio,P/S Percentile,P/B Ratio,P/B Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/Gross Profit (EV/GP),EV/GP Percentile,RV Score,Number of Shares to Buy
0,AXON,748.60,120.741935,100.0,26.177702,100.0,22.801619,91.071429,499.130,100.0,41.815055,98.214286,97.857143,8986
1,AMT,208.74,30.742268,87.5,9.524653,91.071429,26.322826,96.428571,22.506,82.142857,19.714529,87.5,88.928571,32227
2,APH,105.49,48.837963,98.214286,6.844237,83.928571,11.173604,82.142857,25.587,85.714286,20.131747,89.285714,87.857143,63769
3,ADSK,308.14,33.566449,89.285714,10.386800,92.857143,25.197483,92.857143,44.405,94.642857,11.323607,62.5,86.428571,21831
4,AAPL,209.05,25.156438,78.571429,7.798686,87.5,46.756878,98.214286,23.081,83.928571,17.167632,80.357143,85.714286,32179
5,AMD,179.51,35.198039,91.071429,10.488560,94.642857,5.011866,60.714286,48.507,96.428571,19.176954,83.928571,85.357143,37474
6,ADI,231.11,30.651194,85.714286,11.680609,96.428571,3.275786,48.214286,27.305,89.285714,20.687513,92.857143,82.5,29107
7,ANET,122.09,12.547790,26.785714,20.619152,98.214286,15.160809,85.714286,44.193,92.857143,29.548407,96.428571,80.0,55099
8,ADP,310.94,28.605336,82.142857,6.247597,75.0,21.576572,89.285714,21.343,80.357143,13.000972,69.642857,79.285714,21634
9,AVB,196.31,35.435018,92.857143,9.351211,89.285714,2.342490,33.928571,20.432,78.571429,19.610342,85.714286,76.071429,34267


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [23]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_df.to_excel(writer, sheet_name='Value Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [24]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [25]:
column_formats = {
        'A': ['Ticker', string_template],
        'B': ['Price', dollar_template],
        'C': ['Number of Shares to Buy', integer_template],
        'D': ['Price-to-Earnings Ratio', float_template],
        'E': ['PE Percentile', percent_template],
        'F': ['Price-to-Book Ratio', float_template],
        'G': ['PB Percentile',percent_template],
        'H': ['Price-to-Sales Ratio', float_template],
        'I': ['PS Percentile', percent_template],
        'J': ['EV/EBITDA', float_template],
        'K': ['EV/EBITDA Percentile', percent_template],
        'L': ['EV/GP', float_template],
        'M': ['EV/GP Percentile', percent_template],
        'N': ['RV Score', percent_template]
}
for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [26]:
writer.close()